In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from dateutil import parser
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
)
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
df = pd.read_excel('new_test-data.xlsx', sheet_name=1)
df_test = pd.read_excel('April Closed INC.xlsx', sheet_name=1)

In [37]:
df.head()

,Number,Opened,Created,Short description,Category,Caller,Priority,State,Assignment group,Assigned to,Updated,Updated by
0,INC0087058,2022-12-24 20:30:31,2022-12-24 20:39:05,MVW - EHR missing GillRK,MVW,"Gill, Rajwinder (Rajwinder.Gill2@sutterhealth....",2 - High,Closed,Access-Active Directory Domain,"Amer Mohd Jafar, Shaik Mohammad (ShaikMohammad...",2023-01-01 02:00:11,system
1,INC0084178,2022-12-22 20:22:31,2022-12-22 20:37:30,MVW - Unable to Access - Please create Home Dr...,MVW,"Tan, Ralph Curt",2 - High,Closed,Access-Active Directory Domain,"Deshmukh, Satyam (Satyam.Deshmukh@sutterhealth...",2023-01-01 18:00:05,system
2,INC0090247,2022-12-27 11:02:18,2022-12-27 11:07:10,Access- MVW,MVW,"Ona, Marta (Marta.Ona@sutterhealth.org)",4 - Low,Closed,Access-Active Directory Domain,"D B, Krishna (Krishna.DB@sutterhealth.org)",2023-01-03 12:00:13,system
3,INC0094548,2022-12-28 22:46:27,2022-12-28 23:02:36,MVW - GG Missing in AD Acct,MVW,"Bush, Brenda (Brenda.Bush2@sutterhealth.org)",2 - High,Closed,Access-Active Directory Domain,"Deshmukh, Satyam (Satyam.Deshmukh@sutterhealth...",2023-01-05 00:00:27,system
4,INC0096766,2022-12-29 16:01:46,2022-12-29 16:12:42,MVW - unable to access,MVW,"France, Austin",4 - Low,Closed,Access-Active Directory Domain,"Gopakumar, Theertha (Theertha.Gopakumar@sutter...",2023-01-06 12:00:58,system


In [38]:
drop_cols = ["Number", "Opened", "Created", "Caller", "Priority", "State", "Assignment group", "Assigned to", "Updated", "Updated by"]

In [39]:
# df.drop(drop_cols, inplace=True, axis='columns')
df = df[['Short description', 'Category']]

In [25]:
df

,Short description,Category,Updated by
0,MVW - EHR missing GillRK,MVW,system
1,MVW - Unable to Access - Please create Home Dr...,MVW,system
2,Access- MVW,MVW,system
3,MVW - GG Missing in AD Acct,MVW,system
4,MVW - unable to access,MVW,system
...,...,...,...
2077,Active Directory - Email needs to be created f...,AD,system
2078,Request customer be added to the app-powershar...,AD,system
2079,R1 RCM Transition - restrict view in \\dcms2ms...,Shared Drive,system
2080,No AD attached in sailpoint,AD,system


In [ ]:
english_stopwords = stopwords.words('english')
puncuations = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', 
               '>', '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '~']


In [41]:
df.isna().sum()

Short description    0
Category             0
dtype: int64

In [ ]:
df_test.isna().sum()

In [42]:
df.columns = ["SD", "Cat"]

In [43]:
df

,SD,Cat
0,MVW - EHR missing GillRK,MVW
1,MVW - Unable to Access - Please create Home Dr...,MVW
2,Access- MVW,MVW
3,MVW - GG Missing in AD Acct,MVW
4,MVW - unable to access,MVW
...,...,...
2077,Active Directory - Email needs to be created f...,AD
2078,Request customer be added to the app-powershar...,AD
2079,R1 RCM Transition - restrict view in \\dcms2ms...,Shared Drive
2080,No AD attached in sailpoint,AD


In [ ]:
def is_valid_date(word):
    try:
        parser.parse(word)
        return True
    except:
        return False

def clean_row(row):
    porter_stemmer = PorterStemmer()
    row = ' '.join([word.lower() for word in row.split() if word.lower() not in english_stopwords])
    row = ' '.join([word for word in row.split() if word not in puncuations])
    row = ' '.join([word for word in row.split() if not row.isdigit()])
    row = ' '.join([ word for word in row.split() if not is_valid_date(word)])
    # row = ' '.join([ porter_stemmer.stem(word) for word in row.split()])
    row = row.replace("/", "")
    row = row.replace("-", "")
    return row

In [ ]:
df['SD_CLEAN'] = df['SD'].apply(lambda row: clean_row(row))
df_test['SD_CLEAN'] = df_test['Short description'].apply(lambda row: clean_row(row))

In [ ]:
df = df.dropna()

In [ ]:
df['Cat'].unique()

In [ ]:
df

In [ ]:
df['Cat'].value_counts()

In [ ]:
df['Cat'] = df['Cat'].replace(['Shared Folder and Network Drive','Shared Folder or Network Drive'], 'Shared Drive')
df['Cat'] = df['Cat'].replace(['User Account '], 'User Account')
df['Cat'] = df['Cat'].replace(['Sailpoint/AD','Sailpoint '], 'Sailpoint')
df['Cat'] = df['Cat'].replace(['DL'], 'Distribution List')
df['Cat'] = df['Cat'].replace(['Merge Accounts'], 'Account Merge')
df['Cat'] = df['Cat'].replace(['Active Directory'], 'AD')
df['Cat'] = df['Cat'].replace(['Health Plan'], 'Healthplan')
df['Cat'] = df['Cat'].replace(['Shared Mail Box'], 'Shared Mailbox')

In [ ]:
df['Cat'].value_counts()

In [ ]:
df['Cat'].value_counts().plot.bar()

In [ ]:
label_encoder = LabelEncoder()
label = label_encoder.fit_transform(df['Cat'])

In [ ]:
df['Cat_Class'] = label

In [ ]:
final_df = shuffle(df[['SD_CLEAN', 'Cat_Class']], random_state=123)

In [ ]:
final_df.head()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(final_df["SD_CLEAN"], final_df["Cat_Class"], train_size=1, random_state=123)
X_train = final_df['SD_CLEAN']
Y_train = final_df['Cat_Class']
X_test = df_test['SD_CLEAN']

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
# print(y_test.shape)

In [ ]:
X_test

In [ ]:
# creating bag of words 
vec = CountVectorizer()
X_train_cv = vec.fit(X_train)

In [ ]:
len(X_train_cv.vocabulary_)

In [ ]:
X_train_cv_transform = vec.fit_transform(X_train)

In [ ]:
X_test_cv_transform = vec.transform(X_test)

In [ ]:
random_model = RandomForestClassifier(class_weight='balanced')
random_model.fit(X_train_cv_transform.toarray(), Y_train)
y_pred_random_model = random_model.predict(X_test_cv_transform.toarray())

In [ ]:
new_col = label_encoder.inverse_transform(y_pred_random_model)
df_test['Category'] = new_col

In [ ]:
df_test['Category'].value_counts().plot.bar()

In [ ]:
df_test = df_test.drop(['SD_CLEAN'], axis=1)
df_test

In [ ]:
df_test.to_excel('april_predicted_data_closed_INC.xlsx', index=False)